In [1]:
import numpy as np
import time
from joblib import Parallel, delayed
import pickle
from Model import SimplePeerModel, PeerModelwithFeature, MixModel
from Moments import SimplePeerMoments, PeerFeatureMoments, MixMoments
from tqdm import tqdm
import sys

In [2]:
with open('simu_data_collect/model2/training_set.pkl', 'rb') as f:  # data from "set_up.py"
    data = pickle.load(f)
basket_theta = data['basket_theta']
network = data['network']
feature = data['feature']
lb = data['lb']
ub = data['ub']
label_name = data['label_name']
n = network[0].shape[0]
period = len(network)
econmodel = PeerModelwithFeature(n, period)
R = basket_theta.shape[0]

In [3]:
# basket_theta: (R, num_theta)
network_simul, feature_simul = econmodel.get_data(basket_theta[0, :])

In [4]:
network_simul

[<Compressed Sparse Row sparse matrix of dtype 'bool'
 	with 618 stored elements and shape (2511, 2511)>,
 <Compressed Sparse Row sparse matrix of dtype 'bool'
 	with 550 stored elements and shape (2511, 2511)>,
 <Compressed Sparse Row sparse matrix of dtype 'bool'
 	with 606 stored elements and shape (2511, 2511)>,
 <Compressed Sparse Row sparse matrix of dtype 'bool'
 	with 610 stored elements and shape (2511, 2511)>]

In [5]:
feature_simul

array([[-0.26059521],
       [ 0.13972013],
       [ 0.5418499 ],
       ...,
       [-2.25313916],
       [ 0.74209602],
       [-0.30570896]], shape=(2511, 1))

In [6]:
# calculate moments
import numpy as np
from scipy.sparse import csr_matrix, issparse
from scipy.sparse import csr_matrix, tril, isspmatrix
from scipy.sparse import triu as sparse_triu
from Clustering_global import Clustering_global

from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import shortest_path
import time

In [7]:
n = network[0].shape[0] # 2511
period = len(network) # 4

In [8]:
draw1 = (csr_matrix(np.random.rand(n, n) < 0.03)).astype(bool) # 生成随机稀疏矩阵
draw1 = draw1 + draw1.T  # 对称
moment1_list, moment2_list = [], []

In [9]:
start_time = time.time()
for p in range(period):
    Y = network[p]
    deg = np.array(Y.sum(axis=1).A1).flatten()  # 求和+转成 长度 n 的一维 numpy array
    moment1 = np.array([
        np.mean(deg),
        np.var(deg),
        Clustering_global(Y)[0],
    ])
    moment1_list.append(moment1)

print(f" Done. Time spent: {time.time() - start_time:.2f} seconds")

 Done. Time spent: 0.03 seconds


In [10]:
moment1_list

[array([1.13659896, 2.00005837, 0.00334076]),
 array([1.36121067, 2.56367261, 0.00628272]),
 array([1.54201513, 3.07818296, 0.0079722 ]),
 array([1.71007567, 3.65867586, 0.00871137])]

In [11]:
def Stat(*args):
    output_list = []
    for data in args:
        data = np.array(data)
        u = np.mean(data, axis=0)
        v = np.cov(data, rowvar=False)
        v_flat = v[np.triu_indices(v.shape[0])]
        output_list.append(np.hstack([u, v_flat]))
    output = np.hstack(output_list)
    return output

In [12]:
# --- moment2: cross-period statistics ---
start_time = time.time()
for p in range(1, period):
    Y0 = network[p - 1].toarray() if isspmatrix(network[p - 1]) else network[p - 1]
    Y = network[p].toarray() if isspmatrix(network[p]) else network[p]
    deg = np.sum(Y0, axis=1)
    log_deg = np.log1p(deg)
    log_deg_sum = log_deg[:, None] + log_deg[None, :]

    adj = csr_matrix(Y0)
    dist = shortest_path(adj, method='D', directed=False)  # shape = (n, n)
    dist = 1 - 1 / (1 + dist)

    # lower triangular indices
    i1 = np.tril_indices(n, k=-1)
    D1 = np.column_stack([Y0[i1], dist[i1], log_deg_sum[i1]])

    A2 = Y.astype(int)
    B2 = draw1.toarray().astype(int) if isspmatrix(draw1) else draw1.astype(int)
    i2 = np.tril((A2-B2).astype(bool), k=-1)
    D2 = np.column_stack([Y0[i2], dist[i2], log_deg_sum[i2]])

    moment2 = Stat(D1, D2)
    moment2_list.append(moment2)

print(f" Done. Time spent: {time.time() - start_time:.2f} seconds")

 Done. Time spent: 4.99 seconds


In [13]:
moment2_list

[array([ 4.52828273e-04,  9.82267581e-01,  1.16588055e+00,  4.52623363e-04,
        -2.18384465e-04,  6.11683777e-04,  1.76770676e-03, -1.91681442e-02,
         6.60214253e-01,  6.99155143e-03,  9.79041437e-01,  1.17654187e+00,
         6.94270669e-03, -3.34926072e-03,  9.46392060e-03,  3.27447284e-03,
        -2.33560189e-02,  6.72153929e-01]),
 array([ 5.42315009e-04,  9.72403094e-01,  1.33785969e+00,  5.42021076e-04,
        -2.56191369e-04,  7.22311493e-04,  2.69466787e-03, -2.75228263e-02,
         7.27124406e-01,  8.39233919e-03,  9.68626210e-01,  1.34975793e+00,
         8.32195221e-03, -3.93289108e-03,  1.11701280e-02,  4.42021226e-03,
        -3.22491544e-02,  7.39965783e-01]),
 array([ 6.14348659e-04,  9.63412600e-01,  1.46770842e+00,  6.13971429e-04,
        -2.84697000e-04,  8.09877562e-04,  3.38580396e-03, -3.37250282e-02,
         7.69535185e-01,  9.36959011e-03,  9.59305745e-01,  1.48015070e+00,
         9.28185034e-03, -4.30352949e-03,  1.23958138e-02,  5.23420377e-03,
